In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [49]:
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt



7[Files: 0  Bytes: 0  [0 B/s] Re]87[https://raw.githubusercontent.]87input.txt.5            3% [>                             ]   15.06K    --.-KB/s87input.txt.5          100% [=============================>]  424.87K    4.16MB/s87[Files: 1  Bytes: 424.87K [501.]8

In [50]:
with open('input.txt', 'r') as file:
    text = file.read()

print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [51]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [52]:
decode(encode("Hi, I am Pranav"))

'Hi, I am Pranav'

In [53]:
torch.tensor(encode("Hi, I am Pranav"))
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


In [54]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [55]:
BLOCK_SIZE = 32
BATCH_SIZE = 8
eval_interval = 20
max_iters = 10000
eval_iters = 100

In [56]:
torch.manual_seed(1337)

In [57]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    return x, y

In [58]:
xb, yb = get_batch('train')
xb.shape, xb

(torch.Size([8, 32]),
 tensor([[58, 53,  1, 41, 53, 56, 56, 59, 54, 58,  1, 39,  1, 51, 39, 52,  5, 57,
           1, 61, 47, 44, 43,  1, 47, 57,  0, 61, 46, 43, 52,  1],
         [49,  1, 39, 52,  1, 53, 39, 58, 46,  1, 40, 63,  1, 20, 47, 51,  6,  0,
          32, 46, 43,  1, 59, 52, 47, 58, 63,  1, 58, 46, 43,  1],
         [59, 50, 42,  1, 58, 46, 53, 59,  1, 61, 43, 56, 58,  1, 57, 53,  1, 58,
          53, 53,  2,  0,  0, 24, 33, 15, 21, 27, 10,  0, 35, 43],
         [ 8,  0,  0, 35, 13, 30, 35, 21, 15, 23, 10,  0, 28, 56, 53, 60, 43,  1,
          47, 58,  6,  1, 20, 43, 52, 56, 63,  6,  1, 39, 52, 42],
         [58,  1, 57, 46, 43,  8,  0,  0, 32, 30, 13, 26, 21, 27, 10,  0, 18, 53,
          56,  1, 61, 46, 39, 58,  1, 56, 43, 39, 57, 53, 52,  6],
         [56, 61, 47, 41, 49,  6,  1, 50, 43, 58,  1, 47, 58,  1, 40, 43, 11,  0,
          18, 53, 56,  1, 47, 52,  1, 58, 46, 63,  1, 57, 46, 53],
         [25, 10,  0, 35, 47, 58, 46, 42, 56, 39, 61,  1, 63, 53, 59,  1, 46, 43,
  

In [59]:
@torch.no_grad()
def estimate_loss(model: nn.Module):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [60]:
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        if targets is not None:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        else:
            loss = None
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(logits)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([256, 65])
tensor([[-0.2622,  1.7675, -0.6312,  ...,  0.3641, -0.8859, -0.6271],
        [ 0.5739,  0.6314,  0.8301,  ...,  0.0139,  0.0031, -0.3233],
        [-1.9560, -0.8003, -0.5045,  ..., -1.5630,  1.1346, -0.0824],
        ...,
        [-1.4726,  1.1064, -0.0257,  ..., -0.2800, -0.0915,  0.4282],
        [-0.5024, -0.2026, -1.5671,  ...,  0.1108,  0.3925,  0.8353],
        [ 0.2433, -0.1318, -0.6623,  ..., -1.8573,  0.1714, -1.0309]],
       grad_fn=<ViewBackward0>)

&rEnLTjLDJIcLVR'JIHDTHdhsV
v
wxh,nhUYZzAEOZHpgo3q3ZYZes$zuGw,;eMk QqACRfCLgxiW3.O!zDLgA YsTb!dHb!;pK


In [61]:
optim = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [62]:
for iter in range(max_iters):

    if iter % eval_interval == 0:
        losses = estimate_loss(m)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()

step 0: train loss 4.7053, val loss 4.7108
step 20: train loss 4.6721, val loss 4.6900
step 40: train loss 4.6539, val loss 4.6642
step 60: train loss 4.6386, val loss 4.6436
step 80: train loss 4.6073, val loss 4.6112
step 100: train loss 4.5850, val loss 4.6017
step 120: train loss 4.5667, val loss 4.5692
step 140: train loss 4.5350, val loss 4.5550
step 160: train loss 4.5159, val loss 4.5371
step 180: train loss 4.5007, val loss 4.5133
step 200: train loss 4.4696, val loss 4.4941
step 220: train loss 4.4498, val loss 4.4634
step 240: train loss 4.4321, val loss 4.4425
step 260: train loss 4.4067, val loss 4.4314
step 280: train loss 4.3935, val loss 4.4021
step 300: train loss 4.3649, val loss 4.3915
step 320: train loss 4.3575, val loss 4.3612
step 340: train loss 4.3214, val loss 4.3409
step 360: train loss 4.3143, val loss 4.3125
step 380: train loss 4.2870, val loss 4.3090
step 400: train loss 4.2636, val loss 4.2871
step 420: train loss 4.2428, val loss 4.2653
step 440: train 

In [ ]:
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


QUCARDUK:
Burrelam kenstr'd nt ay, an't th f en ane be tof f s, ard eaint-be.
PUR:Por ghy.
AMUE:
vee
